In [1]:
import tensorflow as tf

In [2]:
from songs import *

import copy
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))
        
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('basic midi/track (' + str(i) + ').mid'))
    
all_songs = np.array([song.notes for song in Songs])

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [3]:
def generate():
    X = np.random.randint(0, 2, (128, 128+64, 13))
    X[:, 128:, :] = 0
    Y = np.zeros((128, 128+64, 13))
    Y[:, 64:, :] = X[:, :128, :]
    return X, Y

In [13]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 13
timesteps = 128+64 # timesteps
num_hidden = 50 # hidden layer num of features
history_size = 80

# tf Graph input
X = tf.placeholder("float", [batch_size, timesteps, num_input])
Y = tf.placeholder("float", [batch_size, timesteps, num_input])

loss_op = tf.constant(0.0)

read_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=None)
read_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

read_w = tf.Variable(tf.random_normal([num_hidden, history_size]))
read_b = tf.Variable(tf.random_normal([history_size]))

history = tf.zeros([batch_size, num_input, history_size])

_X = tf.unstack(X, timesteps, 1)
_Y = tf.unstack(Y, timesteps, 1)
for t, inp, truth in zip(np.arange(timesteps), _X, _Y):
    read_lstm_output, read_lstm_state = read_lstm(inp, read_lstm_state)
    
    read = tf.sigmoid(tf.matmul(read_lstm_output, read_w) + read_b)
    read_proba = tf.nn.softmax(read, 1)
    memory_retrieve = tf.matmul(history, tf.expand_dims(read, 2))
    
    if t > 64:
        loss_op += tf.reduce_mean(tf.squared_difference(memory_retrieve, tf.expand_dims(truth, 2)))
    
    history = tf.concat([history[:, :, 1:], tf.expand_dims(inp, 2)], axis=2)

# Define loss and optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = tf.contrib.layers.optimize_loss(
    loss_op, tf.contrib.framework.get_global_step(), optimizer=optimizer,
    learning_rate=learning_rate)

init = tf.global_variables_initializer()

In [14]:
# Start training
sess = tf.Session()
# Run the initializer
sess.run(init)

for step in range(0, training_steps):
    batch_x, batch_y = generate()
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    batch_y = batch_y.reshape((batch_size, timesteps, num_input))
    
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    if step % display_step == 0:
        # Calculate batch loss and accuracy
        L = sess.run(loss_op, feed_dict={X: batch_x, Y: batch_y})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(L))

print("Optimization Finished!")

Step 0, Minibatch Loss= 11650.2852
Step 200, Minibatch Loss= 53.1417
Step 400, Minibatch Loss= 51.6260
Step 600, Minibatch Loss= 50.6630
Step 800, Minibatch Loss= 50.2159
Step 1000, Minibatch Loss= 50.2410
Step 1200, Minibatch Loss= 49.9679
Step 1400, Minibatch Loss= 28.3191
Step 1600, Minibatch Loss= 25.3209
Step 1800, Minibatch Loss= 24.8480
Step 2000, Minibatch Loss= 24.7911
Step 2200, Minibatch Loss= 24.5198
Step 2400, Minibatch Loss= 24.5902
Step 2600, Minibatch Loss= 24.4773
Step 2800, Minibatch Loss= 24.3188
Step 3000, Minibatch Loss= 24.2090
Step 3200, Minibatch Loss= 24.2101
Step 3400, Minibatch Loss= 24.0854
Step 3600, Minibatch Loss= 24.3606
Step 3800, Minibatch Loss= 23.9981
Step 4000, Minibatch Loss= 24.1131
Step 4200, Minibatch Loss= 24.0510
Step 4400, Minibatch Loss= 24.0250
Step 4600, Minibatch Loss= 24.0414
Step 4800, Minibatch Loss= 23.8652
Step 5000, Minibatch Loss= 23.9796
Step 5200, Minibatch Loss= 23.9045
Step 5400, Minibatch Loss= 23.8742
Step 5600, Minibatch Los

In [15]:
mr = sess.run(read_proba, feed_dict={X: batch_x, Y: batch_y})

In [16]:
mr

array([[ 0.01198789,  0.0119833 ,  0.01198484,  0.01198391,  0.01200021,
         0.01198176,  0.01198181,  0.01198177,  0.01198416,  0.01199307,
         0.01198212,  0.01198317,  0.01198664,  0.01198253,  0.01198177,
         0.01198247,  0.03219713,  0.01199065,  0.01198539,  0.0119842 ,
         0.0119827 ,  0.01198525,  0.01198639,  0.01198911,  0.01198392,
         0.0119818 ,  0.01198243,  0.01198211,  0.01198255,  0.01200018,
         0.01198619,  0.01198251,  0.01198284,  0.01198749,  0.01198289,
         0.01198324,  0.01198457,  0.01199417,  0.01198344,  0.0119818 ,
         0.01198644,  0.01198936,  0.01198176,  0.01200871,  0.01198359,
         0.01198215,  0.01198291,  0.0119919 ,  0.0119833 ,  0.01198239,
         0.01199857,  0.01198762,  0.01198439,  0.01198534,  0.0119936 ,
         0.01201394,  0.0119821 ,  0.01198208,  0.01198266,  0.01199127,
         0.01198565,  0.01198227,  0.01198475,  0.01201481,  0.01209407,
         0.01198574,  0.01200409,  0.01203482,  0.0